se instalan y cargan las librerias necesarias

In [1]:
pip install gensim


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pprint
import pandas as pd
import gensim
import re
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.stem import *
import numpy as np
np.random.seed(2018)
import nltk
import nltk.stem as stemmer
stemmer = PorterStemmer()
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rober\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

se carga el texto y se genera un dataframe de los titulos limpios

In [3]:
File = pd.ExcelFile('DatosWoS.xlsx')
df=File.parse('Sheet1')



In [4]:
df.describe()

,Año,referencias
count,250.000000,250.000000
mean,2016.424000,25.544000
std,3.133608,21.261049
min,2010.000000,0.000000
25%,2014.000000,11.000000
50%,2017.000000,18.000000
75%,2019.000000,36.000000
max,2022.000000,161.000000


In [5]:
titles = df["Titulo"]

In [6]:
text_corpus = titles.values.tolist()


In [7]:
for i in range(len(text_corpus)):
    text_corpus[i] = str(text_corpus[i])
    text_corpus[i] = re.sub("[!,*)@#%(&$_?.^:-]'", '', text_corpus[i])
    text_corpus[i] = text_corpus[i].replace('"','')
    text_corpus[i] = text_corpus[i].lower()
    text_corpus[i] = text_corpus[i].replace("movie","")
    text_corpus[i] = text_corpus[i].replace("movies","")


se genera un dataframe de index y luego se fusiona con el de titulos, luego se tockeniza el texto

In [8]:
indexs = []
for i in range(len(text_corpus)):
    indexs.append(i)

In [9]:
df2 = pd.DataFrame(list(zip(text_corpus, indexs)),
               columns =['Titulo', 'index'])

In [10]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [11]:
documents = df2
doc_sample = documents[documents['index'] == 0].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['', 'recommendation', 'system', 'based', 'on', '', 'swarm']


 tokenized and lemmatized document: 
['recommend', 'base', 'swarm']


In [12]:
processed_docs = documents['Titulo'].map(preprocess)
processed_docs[:10]

0                           [recommend, base, swarm]
1    [dynam, effect, rat, revenu, viewer, satisfact]
2                                         [descript]
3                     [success, predict, data, mine]
4                                                 []
5                [exploit, charact, network, summar]
6     [tradit, cultur, chines, case, shoot, huizhou]
7                    [feng, xiaogang, economi, oper]
8                [summar, charact, network, analysi]
9    [attribut, trailer, constitut, purchas, intent]
Name: Titulo, dtype: object

se genera el diccionario con el bag of words que se obtuvo de la tockenizacion

In [13]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 base
1 recommend
2 swarm
3 dynam
4 effect
5 rat
6 revenu
7 satisfact
8 viewer
9 descript
10 data


In [14]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)


se prepara y entrena la red lda testeando con el titulo nº52

In [15]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[52]

[]

In [16]:
bow_doc_52 = bow_corpus[52]
for i in range(len(bow_doc_52)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_52[i][0], 
                                               dictionary[bow_doc_52[i][0]], 
bow_doc_52[i][1]))

In [17]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.6751532289867246), (1, 0.7376775158487614)]


In [18]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [19]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.741*"recommend" + 0.246*"base" + 0.004*"analysi" + 0.004*"predict" + 0.004*"learn"
Topic: 1 
Words: 0.904*"base" + 0.037*"analysi" + 0.027*"recommend" + 0.027*"predict" + 0.006*"learn"
Topic: 2 
Words: 0.864*"analysi" + 0.100*"recommend" + 0.027*"base" + 0.005*"predict" + 0.005*"learn"
Topic: 3 
Words: 0.431*"predict" + 0.276*"base" + 0.276*"learn" + 0.009*"analysi" + 0.009*"recommend"
Topic: 4 
Words: 0.458*"learn" + 0.260*"predict" + 0.157*"recommend" + 0.110*"base" + 0.015*"analysi"
Topic: 5 
Words: 0.460*"recommend" + 0.410*"base" + 0.114*"predict" + 0.008*"analysi" + 0.008*"learn"
Topic: 6 
Words: 0.473*"analysi" + 0.189*"base" + 0.132*"recommend" + 0.132*"predict" + 0.075*"learn"
Topic: 7 
Words: 0.519*"base" + 0.359*"recommend" + 0.071*"predict" + 0.041*"learn" + 0.010*"analysi"
Topic: 8 
Words: 0.605*"learn" + 0.167*"base" + 0.163*"predict" + 0.060*"recommend" + 0.005*"analysi"
Topic: 9 
Words: 0.701*"predict" + 0.203*"analysi" + 0.075*"recommend" + 0.010*"ba

In [20]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.882*"learn" + 0.057*"recommend" + 0.046*"predict" + 0.007*"analysi" + 0.007*"base"
Topic: 1 Word: 0.698*"base" + 0.245*"recommend" + 0.025*"predict" + 0.016*"analysi" + 0.016*"learn"
Topic: 2 Word: 0.463*"analysi" + 0.206*"learn" + 0.203*"predict" + 0.064*"recommend" + 0.064*"base"
Topic: 3 Word: 0.517*"analysi" + 0.244*"recommend" + 0.080*"base" + 0.080*"predict" + 0.080*"learn"
Topic: 4 Word: 0.861*"recommend" + 0.074*"base" + 0.029*"analysi" + 0.028*"learn" + 0.007*"predict"
Topic: 5 Word: 0.665*"predict" + 0.176*"recommend" + 0.105*"learn" + 0.040*"base" + 0.013*"analysi"
Topic: 6 Word: 0.797*"predict" + 0.151*"recommend" + 0.033*"learn" + 0.009*"analysi" + 0.009*"base"
Topic: 7 Word: 0.428*"analysi" + 0.191*"predict" + 0.178*"learn" + 0.136*"recommend" + 0.066*"base"
Topic: 8 Word: 0.508*"analysi" + 0.123*"recommend" + 0.123*"base" + 0.123*"predict" + 0.123*"learn"
Topic: 9 Word: 0.978*"analysi" + 0.006*"recommend" + 0.006*"base" + 0.005*"learn" + 0.005*"predict"


In [21]:
processed_docs[52]

['novel', 'method', 'measur', 'reliabl', 'bollywood', 'rat']

In [22]:
for index, score in sorted(lda_model[bow_corpus[52]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.10000000149011612	 
Topic: 0.741*"recommend" + 0.246*"base" + 0.004*"analysi" + 0.004*"predict" + 0.004*"learn"

Score: 0.10000000149011612	 
Topic: 0.904*"base" + 0.037*"analysi" + 0.027*"recommend" + 0.027*"predict" + 0.006*"learn"

Score: 0.10000000149011612	 
Topic: 0.864*"analysi" + 0.100*"recommend" + 0.027*"base" + 0.005*"predict" + 0.005*"learn"

Score: 0.10000000149011612	 
Topic: 0.431*"predict" + 0.276*"base" + 0.276*"learn" + 0.009*"analysi" + 0.009*"recommend"

Score: 0.10000000149011612	 
Topic: 0.458*"learn" + 0.260*"predict" + 0.157*"recommend" + 0.110*"base" + 0.015*"analysi"

Score: 0.10000000149011612	 
Topic: 0.460*"recommend" + 0.410*"base" + 0.114*"predict" + 0.008*"analysi" + 0.008*"learn"

Score: 0.10000000149011612	 
Topic: 0.473*"analysi" + 0.189*"base" + 0.132*"recommend" + 0.132*"predict" + 0.075*"learn"

Score: 0.10000000149011612	 
Topic: 0.519*"base" + 0.359*"recommend" + 0.071*"predict" + 0.041*"learn" + 0.010*"analysi"

Score: 0.10000000149011

In [23]:
for index, score in sorted(lda_model_tfidf[bow_corpus[52]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.10000000149011612	 
Topic: 0.882*"learn" + 0.057*"recommend" + 0.046*"predict" + 0.007*"analysi" + 0.007*"base"

Score: 0.10000000149011612	 
Topic: 0.698*"base" + 0.245*"recommend" + 0.025*"predict" + 0.016*"analysi" + 0.016*"learn"

Score: 0.10000000149011612	 
Topic: 0.463*"analysi" + 0.206*"learn" + 0.203*"predict" + 0.064*"recommend" + 0.064*"base"

Score: 0.10000000149011612	 
Topic: 0.517*"analysi" + 0.244*"recommend" + 0.080*"base" + 0.080*"predict" + 0.080*"learn"

Score: 0.10000000149011612	 
Topic: 0.861*"recommend" + 0.074*"base" + 0.029*"analysi" + 0.028*"learn" + 0.007*"predict"

Score: 0.10000000149011612	 
Topic: 0.665*"predict" + 0.176*"recommend" + 0.105*"learn" + 0.040*"base" + 0.013*"analysi"

Score: 0.10000000149011612	 
Topic: 0.797*"predict" + 0.151*"recommend" + 0.033*"learn" + 0.009*"analysi" + 0.009*"base"

Score: 0.10000000149011612	 
Topic: 0.428*"analysi" + 0.191*"predict" + 0.178*"learn" + 0.136*"recommend" + 0.066*"base"

Score: 0.10000000149011

se realiza un test con un titulo no visto por la red lda

In [24]:
unseen_document = 'A theoretical analysis of time dependent fragment momenta in indirect photofragmentation'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.5499798655509949	 Topic: 0.864*"analysi" + 0.100*"recommend" + 0.027*"base" + 0.005*"predict" + 0.005*"learn"
Score: 0.050015389919281006	 Topic: 0.473*"analysi" + 0.189*"base" + 0.132*"recommend" + 0.132*"predict" + 0.075*"learn"
Score: 0.05000433698296547	 Topic: 0.701*"predict" + 0.203*"analysi" + 0.075*"recommend" + 0.010*"base" + 0.010*"learn"
Score: 0.050000354647636414	 Topic: 0.904*"base" + 0.037*"analysi" + 0.027*"recommend" + 0.027*"predict" + 0.006*"learn"
Score: 0.05000000819563866	 Topic: 0.741*"recommend" + 0.246*"base" + 0.004*"analysi" + 0.004*"predict" + 0.004*"learn"
Score: 0.05000000819563866	 Topic: 0.431*"predict" + 0.276*"base" + 0.276*"learn" + 0.009*"analysi" + 0.009*"recommend"
Score: 0.05000000819563866	 Topic: 0.458*"learn" + 0.260*"predict" + 0.157*"recommend" + 0.110*"base" + 0.015*"analysi"
Score: 0.05000000819563866	 Topic: 0.460*"recommend" + 0.410*"base" + 0.114*"predict" + 0.008*"analysi" + 0.008*"learn"
Score: 0.05000000819563866	 Topic: 0.51